In [1]:
## hide warning
import warnings
warnings.filterwarnings("ignore")

In [30]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType

FILE_PATH = r"..\00 Dataset\docs\facebook\Meta-09-30-2024-Exhibit-99-1_FINAL.pdf"

loader = DoclingLoader(file_path=FILE_PATH,
                       export_type=ExportType.MARKDOWN,
                       )

In [31]:
docs = loader.load()

INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importin

In [33]:
# print(docs[0].page_content)
# write markdown to file
with open("output.md", "w", encoding="utf-8") as f:
    f.write(docs[0].page_content)
    f.close()

### PDF to MarkDown with Images and Tables

In [36]:
import logging
import time
from pathlib import Path
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat

_log = logging.getLogger(__name__)
IMAGE_RESOLUTION_SCALE = 2.0

def main():
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path('localfile path')
    input_doc_path = r"..\00 Dataset\docs\facebook\Earnings-Presentation-Q3-2024.pdf"
    output_dir = Path("scratch")

    # Important: For operating with page images, we must keep them, otherwise the DocumentConverter
    # will destroy them for cleaning up memory.
    # This is done by setting PdfPipelineOptions.images_scale, which also defines the scale of images.
    # scale=1 correspond of a standard 72 DPI image
    # The PdfPipelineOptions.generate_* are the selectors for the document elements which will be enriched
    # with the image field
    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem

    # Save page images
    for page_no, page in conv_res.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="PNG")

    # Save images of figures and tables
    table_counter = 0
    picture_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-table-{table_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-picture-{picture_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

    # Save markdown with externally referenced pictures
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)

    end_time = time.time() - start_time

    _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")

main()

INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importin

### Describe Image with LLM

In [22]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
import base64

import httpx

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data_weather = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

# read image from file
with open(r"scratch\Earnings-Presentation-Q3-2024-with-image-refs_artifacts\image_000004_8139d1246423312e74e78335d99a132a935f7de1b62a31df96dc7d8c91a47a7a.png", "rb") as f:
    image_data = base64.b64encode(f.read()).decode("utf-8")

with open(r"scratch\Earnings-Presentation-Q3-2024-with-image-refs_artifacts\image_000000_3db7e1eab3213fbdaaf83be4c917fbb9fd97bdf42a5033bd7e7e4df4b53f7b18.png", "rb") as f:
    image_data1 = base64.b64encode(f.read()).decode("utf-8")

model = ChatOpenAI(model="gpt-4o-mini")

system = SystemMessage("""
                Extract detailed financial information from the provided image. Start by identifying the company name, document title, and any relevant details from the header and footer.

                Ensure to:
                    - Thoroughly extract all financial figures and metrics mentioned, such as revenue, profit, assets, liabilities, etc.
                    - Explain the financial data with technical details, including any relevant financial terminology or calculation methods.
                    - Summarize any regulatory or legal information provided in the document.
                
                Provide a complete and detailed description of the image, ensuring no important data is missed.
                       """)

message = HumanMessage(
    content=[
        {"type": "text", "text": "Here is some reference content from the image: this is meta earning calls for Q3 2024. <fill this in runtime>"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{image_data}"}

        },
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{image_data1}"}

        },
    ],
)
response = model.invoke([message])
print(response.content)

The image presents a bar chart summarizing Meta's earnings calls for Q3 2024, detailing revenue by region compared to previous quarters. Here’s a breakdown of the data presented:

### Earnings Overview:
- **Q3 2022**: $27,714
- **Q4 2022**: $32,165
- **Q1 2023**: $28,645
- **Q2 2023**: $31,999
- **Q3 2023**: $34,146
- **Q4 2023**: $40,111
- **Q1 2024**: $36,455
- **Q2 2024**: $39,071
- **Q3 2024**: $40,589

### Revenue Breakdown by Region (For Q3 2024):
- **Rest of World**: $5,268
- **Asia-Pacific**: $8,220
- **Europe**: $9,492
- **US & Canada**: $17,609

This data provides insights into the growth trends and performance of Meta across different regions over several quarters.


### MarkDown Splitters

In [24]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, MarkdownTextSplitter

markdown_path = r"scratch\Earnings-Presentation-Q3-2024-with-image-refs.md"
with open(markdown_path, "r", encoding="utf-8") as f:
    markdown_content = f.read()

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)
md_header_splits = markdown_splitter.split_text(markdown_content)



In [27]:
import re

def extract_image_urls_and_clean_content(page_content):
    # Define the regex pattern to match image URLs
    pattern = r"!\[Image\]\(([^)]+)\)"

    # Find all matches in the page content
    image_urls = re.findall(pattern, page_content)

    # Remove all matched image URLs from the content
    cleaned_content = re.sub(pattern, "", page_content)

    return image_urls, cleaned_content

In [28]:
for page in md_header_splits:
    # print(f"Header: {page.metadata}")
    print(f"Content: {page.page_content}")
    # Extract image URLs from the page content
    image_urls, cleaned_content = extract_image_urls_and_clean_content(page.page_content)
    print(f"Image URLs: {image_urls}")
    print(f"Cleaned Content: {cleaned_content}")
    print("\n")

# md_header_splits

Content: ## Meta Earnings Presentation Q3 2024  
![Image](Earnings-Presentation-Q3-2024-with-image-refs_artifacts%5Cimage_000000_3db7e1eab3213fbdaaf83be4c917fbb9fd97bdf42a5033bd7e7e4df4b53f7b18.png)
Image URLs: ['Earnings-Presentation-Q3-2024-with-image-refs_artifacts%5Cimage_000000_3db7e1eab3213fbdaaf83be4c917fbb9fd97bdf42a5033bd7e7e4df4b53f7b18.png']
Cleaned Content: ## Meta Earnings Presentation Q3 2024  



Content: ## Advertising Revenue by User Geography  
![Image](Earnings-Presentation-Q3-2024-with-image-refs_artifacts%5Cimage_000001_278dad5e9c44f5552629e781407508746b2c8c43e8597f0a69705fe8f19aa660.png)  
In Millions  
![Image](Earnings-Presentation-Q3-2024-with-image-refs_artifacts%5Cimage_000002_1251d14efb5cba3ebd6d19703f740b8b0bb6b1d1a99275c9eb52941cb5e4c0f1.png)  
Our revenue by user geography is geographically apportioned based on our estimation of the geographic location of our users when they perform a revenue-generating activity. This allocation differs from our revenue d